# Installing Modules and Pre-requisites

In [23]:
import yfinance as yf
import pandas as pd
import numpy as np

# Importing Data

In [66]:
ticker = "SPY"
data = yf.download(ticker, start="2010-01-01", end="2025-05-20", auto_adjust=False)


[*********************100%***********************]  1 of 1 completed


In [68]:
data["daily_return"] = data["Adj Close"].pct_change()
data["daily_return"].fillna(0, inplace=True)  # Replace first NaN with 0
data["state"] = np.where(data["daily_return"] >= 0, "up", "down")
data

/tmp/ipython-input-68-3433722459.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["daily_return"].fillna(0, inplace=True)  # Replace first NaN with 0


Price,Adj Close,Close,High,Low,Open,Volume,daily_return,state
Ticker,SPY,SPY,SPY,SPY,SPY,SPY,,
Date,,,,,,,,
2010-01-04,85.515617,113.330002,113.389999,111.510002,112.370003,118944600,NaN,down
2010-01-05,85.741982,113.629997,113.680000,112.849998,113.260002,111579900,0.002647,up
2010-01-06,85.802361,113.709999,113.989998,113.430000,113.519997,116074400,0.000704,up
2010-01-07,86.164558,114.190002,114.330002,113.180000,113.500000,131091100,0.004221,up
2010-01-08,86.451324,114.570000,114.620003,113.660004,113.889999,126402800,0.003328,up
...,...,...,...,...,...,...,...,...
2025-05-13,585.110291,586.840027,589.080017,582.840027,583.409973,67947200,0.006604,up
2025-05-14,585.858093,587.590027,588.979980,585.539978,587.809998,66283500,0.001278,up


# Finding Up Day or Down Day

In [74]:
up_counts = len(data[data["state"] == "up"])

In [75]:
down_counts = len(data[data["state"] == "down"])

# Calculating Transition Matrix to find probabilities

In [77]:
# Calculate transition counts and probabilities
up_to_up = len(data[(data["state"] == "up") & (data["state"].shift(-1) == "up")]) / (data["state"] == "up").sum()
down_to_up = len(data[(data["state"] == "up") & (data["state"].shift(-1) == "down")]) / (data["state"] == "up").sum()
up_to_down = len(data[(data["state"] == "down") & (data["state"].shift(-1) == "up")]) / (data["state"] == "down").sum()
down_to_down = len(data[(data["state"] == "down") & (data["state"].shift(-1) == "down")]) / (data["state"] == "down").sum()

# Construct the transition matrix
transition_matrix = pd.DataFrame({
    "up": [up_to_up, up_to_down],
    "down": [down_to_up, down_to_down]
}, index=["up", "down"])

print(transition_matrix)


            up      down
up    0.547608  0.451928
down  0.567930  0.432070


In [78]:
len(data[(data["state"] == "up") & (data["state"].shift(-1) == "down") & (data["state"].shift(-2) == "down") & (data["state"].shift(-3) == "down") & (data["state"].shift(-4) == "down") & (data["state"].shift(-5) == "down")]) / len(data[(data["state"].shift(1) == "down") & (data["state"].shift(2) == "down") & (data["state"].shift(3) == "down") & (data["state"].shift(4) == "down") & (data["state"].shift(5) == "down")])

0.673469387755102

📉📉📉📉📉 ➡️ 📈
**It’s calculating the probability that a green candle (📈) comes right after 5 consecutive red candles (📉).**